In [1]:
### Import Statements ###
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.svm import SVC, NuSVC, LinearSVC
from sklearn.linear_model import LogisticRegressionCV, SGDClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from scipy.sparse import coo_matrix, hstack
import scipy
from sklearn.model_selection import GridSearchCV
import re


C:\Users\HP\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
df = pd.read_csv("~/Documents/AV codefest Sentiment analysis/train.csv")
df2 = pd.read_csv("~/Documents/AV codefest Sentiment analysis/test.csv")
df.head()

,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...
1,2,0,Finally a transparant silicon case ^^ Thanks t...
2,3,0,We love this! Would you go? #talk #makememorie...
3,4,0,I'm wired I know I'm George I was made that wa...
4,5,1,What amazing service! Apple won't even talk to...


In [3]:
df.label.value_counts()/len(df)

0    0.744192
1    0.255808
Name: label, dtype: float64

In [4]:
def remo(features):
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",features[0]).split())

In [5]:
df['tweet'] = df[['tweet']].apply(remo,axis=1)
df2['tweet'] = df2[['tweet']].apply(remo,axis=1)


In [6]:
imbalance_weight = df.label.value_counts(normalize=True)[1]/df.label.value_counts(normalize=True)[0]
imbalance_weight

0.34373939599592807

In [10]:
params = {"tfidf__ngram_range": [(1, 2)],
          "svc__C": [.01, .1, 1, 10, 100]}

clf = Pipeline([("tfidf", TfidfVectorizer(sublinear_tf=True)),
                ("svc", LinearSVC(loss='hinge'))])

gs = GridSearchCV(clf, params, verbose=2, n_jobs=-1)
gs.fit(df['tweet'], df['label'])
print(gs.best_estimator_)
print(gs.best_score_)

Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:   12.4s finished


Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
 ...e', max_iter=1000, multi_class='ovr',
     penalty='l2', random_state=None, tol=0.0001, verbose=0))])
0.8920454545454546


In [12]:
predicted = gs.predict(df2['tweet'])

In [14]:
predicted

array([1, 1, 1, ..., 1, 1, 0], dtype=int64)

In [179]:
#predicted
#test_pred = lreg.predict_proba(test_bow)
#test_pred_int = predicted[:,1] >= 0.015
#test_pred_int = test_pred_int.astype(np.int)


In [180]:
#test_pred_int

array([0, 1, 0, ..., 0, 0, 0])

In [17]:
final_predict = pd.DataFrame(predicted,columns=['label'])
result = pd.DataFrame(df2['id'],columns=['id'])
result = pd.concat([result,final_predict],axis=1)
result.to_csv('final_predictions2.csv',index=False)

In [16]:
result['label'].value_counts()

0    1353
1     600
Name: label, dtype: int64